# 1. Cleaning

In [13]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

df = pd.read_csv('./data/grouped_sales_train.csv')
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month,year-month
0,0,0,32,884.0,6.0,2013-01
1,0,0,33,1041.0,3.0,2013-01
2,0,0,35,247.0,1.0,2013-01
3,0,0,43,221.0,1.0,2013-01
4,0,0,51,257.0,2.0,2013-01


The below concludes that not all shop id's are useful.  Some only have data on a few months and therefore, would be hard to use.  This is when I realized that I should probably focus on the shop id's that are in the test.csv file.

In [5]:
df.loc[(df['date_block_num']==0) & (df['shop_id']==0),]
df.loc[(df['date_block_num']==5) & (df['shop_id']==0),]

,date_block_num,shop_id,item_id,item_price,item_cnt_month,year-month


In [6]:
df.loc[(df['shop_id']==0),]

,date_block_num,shop_id,item_id,item_price,item_cnt_month,year-month
0,0,0,32,884.0,6.0,2013-01
1,0,0,33,1041.0,3.0,2013-01
2,0,0,35,247.0,1.0,2013-01
3,0,0,43,221.0,1.0,2013-01
4,0,0,51,257.0,2.0,2013-01
...,...,...,...,...,...,...
65655,1,0,22138,188.0,1.0,2013-02
65656,1,0,22139,624.0,1.0,2013-02
65657,1,0,22151,1876.0,8.0,2013-02
65658,1,0,22154,1314.0,2.0,2013-02


## Data Manipulation

- Group by date, then shop, then item and get the sum of `item_price` and `item_cnt_day`
- Reset column name to be `item_cnt_month` because you summed by month

In [2]:
grouped = df.groupby(['date_block_num', 'shop_id', 'item_id'], as_index=False)[['item_price', 'item_cnt_day']].sum().copy()
grouped.columns = ['date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_month']

In [6]:
grouped.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_month
0,0,0,32,884.0,6.0
1,0,0,33,1041.0,3.0
2,0,0,35,247.0,1.0
3,0,0,43,221.0,1.0
4,0,0,51,257.0,2.0


Save to csv.

In [10]:
# grouped.to_csv('./data/grouped_sales_train.csv', index=False)

## Read in test data

and figure out which shops I need to predict.

In [7]:
test = pd.read_csv('./data/test.csv')

In [8]:
len(test['shop_id'].unique())

42

In [9]:
test_shop_ids = test['shop_id'].unique().tolist()
print(test_shop_ids)

[5, 4, 6, 3, 2, 7, 10, 12, 28, 31, 26, 25, 22, 24, 21, 15, 16, 18, 14, 19, 42, 50, 49, 53, 52, 47, 48, 57, 58, 59, 55, 56, 36, 37, 35, 38, 34, 46, 41, 44, 39, 45]


In [10]:
print(test['item_id'].unique())
len(test['item_id'].unique())

[ 5037  5320  5233 ... 15757 19648   969]


5100

## Break data out by `shop_id`

Data will be saved in a folder called shops

In [11]:
def shop_id_output(df, shop_ids):
    '''Break out data by shop_id'''
    
    os.mkdir('./data/shops')
    
    for shop in shop_ids:
        shop_df = df.loc[df['shop_id']==shop,]
        shop_df.to_csv(f'./data/shops/shop_{shop}.csv', index=False)

In [12]:
shop_id_output(df, test_shop_ids)

## Check to see if there are different items in train verses test dataset

In [56]:
def item_check(train, test, shop_ids):
    '''Check to see if there are any missing items in train dataset compared to test dataset'''
    
    items_dict = {}
    items_list = []
    
    for shop in shop_ids:
        test_item_list = set(test.loc[test['shop_id']==shop,'item_id'])
        train_item_list = set(train.loc[train['shop_id']==shop,'item_id'])
        
        if not all(item in train_item_list for item in test_item_list):
            for item in test_item_list:
                if item not in train_item_list:
                    items_list.append(item)
        
        items_dict[shop] = len(items_list)
        items_list = []
    
    return items_dict

In [57]:
items_not_in_test = item_check(df, test, test_shop_ids)

In [60]:
len(items_not_in_test.keys())

42

In [67]:
print(items_not_in_test)

{5: 2501, 4: 2626, 6: 1638, 3: 2981, 2: 3187, 7: 2398, 10: 2980, 12: 2483, 28: 1480, 31: 1178, 26: 2122, 25: 1087, 22: 2329, 24: 2337, 21: 1819, 15: 2370, 16: 2317, 18: 2241, 14: 2603, 19: 2100, 42: 1396, 50: 2316, 49: 2967, 53: 2424, 52: 2333, 47: 2217, 48: 2653, 57: 1682, 58: 2335, 59: 2587, 55: 4701, 56: 1931, 36: 4848, 37: 2518, 35: 2153, 38: 2463, 34: 3839, 46: 2160, 41: 2751, 44: 2309, 39: 2847, 45: 2589}


In [75]:
# Shop 55

test_item_list = set(test.loc[test['shop_id']==55,'item_id'])
train_item_list = set(df.loc[df['shop_id']==55,'item_id'])

print(len(test_item_list))
len(train_item_list)

5100


1985

**Conclusion:**

There are a substantial amount of items missing between the test and train datasets for each shop.  This leads me to believe that a lot of items should be 0 because there will be no prior data on it.

In [88]:
# check unique items for a given month
train_item_list_0 = set(df.loc[(df['date_block_num'] == 30) & (df['shop_id'] == 55),'item_id'])
train_item_list_1 = set(df.loc[(df['date_block_num'] == 32) & (df['shop_id'] == 55),'item_id'])

# comparing the differences in count of items between different periods
print(len(train_item_list_0))
print(len(train_item_list_1))
len(test.loc[test['shop_id']==55,'item_id'])

427
474


5100